<a href="https://colab.research.google.com/github/starminalush/mlops_report/blob/main/ways%20of%20convert%20rubert_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Устанавливаем нужные зависимости

In [1]:
!pip install onnx transformers onnxruntime folium==0.2.1 optimum[onnxruntime]

     |████████████████████████████████| 12.8 MB 4.8 MB/s 
     |████████████████████████████████| 4.0 MB 14.3 MB/s 
     |████████████████████████████████| 5.2 MB 11.4 MB/s 
     |████████████████████████████████| 69 kB 7.4 MB/s 
     |████████████████████████████████| 62 kB 555 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
     |████████████████████████████████| 895 kB 51.9 MB/s 
     |████████████████████████████████| 6.6 MB 33.4 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 325 kB 38.4 MB/s 
     |████████████████████████████████| 1.1 MB 48.7 MB/s 
     |████████████████████████████████| 212 kB 67.0 MB/s 
     |████████████████████████████████| 136 kB 55.3 MB/s 
     |████████████████████████████████| 127 kB 47.4 MB/s 
     |██████

Фиксируем версии библиотек

In [2]:
!pip freeze > req.txt

Импорты

In [3]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
from transformers.onnx import export
from pathlib import Path
from typing import Mapping, OrderedDict
from transformers.onnx import OnnxConfig
from transformers import AutoConfig
import onnxruntime as nxrun
import onnx
import numpy as np
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from torch.nn.utils import prune
from optimum.onnxruntime import ORTQuantizer

Запускаем rubert как есть

In [4]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [5]:
%%time
predict('Правительство выделит 16 миллиардов рублей на поддержку клещей')
#вернулся нейтральный класс

CPU times: user 104 ms, sys: 168 ms, total: 271 ms
Wall time: 421 ms


0

Переводим в ONNX
1. Есть библиотека transforms, где все из коробки

https://huggingface.co/docs/transformers/serialization - делаем все по лучшим гайдам

In [ ]:
class DistilBertOnnxConfig(OnnxConfig):
    @property
    def inputs(self) -> Mapping[str, Mapping[int, str]]:
        return OrderedDict(
            [
                ("input_ids", {0: "batch", 1: "sequence"}),
                ("attention_mask", {0: "batch", 1: "sequence"}),
                ("token_type_ids", {0: "batch", 1: "sequence"}),
            ]
        )

In [ ]:
config = AutoConfig.from_pretrained("blanchefort/rubert-base-cased-sentiment")
onnx_config_for_seq_clf = DistilBertOnnxConfig(config, task="sequence-classification")
print(onnx_config_for_seq_clf.outputs)

OrderedDict([('logits', {0: 'batch'})])


In [ ]:
!mkdir -p output/onnx_transforms

In [ ]:
onnx_inputs, onnx_outputs = export(
        tokenizer,
        model,
        onnx_config_for_seq_clf,
        output=Path("output/onnx_transforms/rubert.onnx"),
        opset=11)

Пробуем запустить в ONNX и посмотреть время инференса

In [ ]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert.onnx", sess_options, providers)

Смотрим на входы и выходы импортированной модели

In [ ]:
input_names = model_ONNX.get_inputs()
for input_name in input_names:
  print(input_name.name)

input_ids
attention_mask
token_type_ids


In [ ]:
for on in model_ONNX.get_outputs():
  print(on.name)

logits


In [ ]:
def predict_onnx(text):
  inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='np')
  result  = model_ONNX.run(None, dict(inputs))
  predicted = result.index(max(result))
  return predicted

In [ ]:
%%time
predict_onnx('Правительство выделит 16 миллиардов рублей на поддержку клещей')

CPU times: user 62 ms, sys: 2.02 ms, total: 64 ms
Wall time: 67.9 ms


0

Квантизация 

Делаем тоже по лучшим гайдам https://github.com/huggingface/optimum

Есть три вида квантизации - статическая, динамическая и Quantization-Aware-Training(QAT)

Динамическая квантизация не требует ничего, поэтому она самая простая

In [ ]:
!mkdir -p output/quantization

In [ ]:
model_checkpoint = "blanchefort/rubert-base-cased-sentiment"
# The type of quantization to apply
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained(model_checkpoint, feature="sequence-classification")

# Quantize the model!
quantizer.export(
    onnx_model_path="output/quantization/rubert.onnx",
    onnx_quantized_model_output_path="output/quantization/rubert-dyn-quantized.onnx",
    quantization_config=qconfig,
)

PosixPath('output/quantization/rubert-dyn-quantized.onnx')

Пробуем запустить динамечески квантизированную ONNX модель и посмотреть на время инференса

In [ ]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("/content/output/quantization/rubert-dyn-quantized.onnx", sess_options, providers)

In [ ]:
%%time
predict_onnx('Правительство выделит 16 миллиардов рублей на поддержку клещей')

CPU times: user 41.1 ms, sys: 2.32 ms, total: 43.4 ms
Wall time: 48 ms


0

Прунинг модели - и снова лучшие мануалы интернета https://github.com/Huffon/nlp-various-tutorials/blob/master/pruning-bert.ipynb


https://aclanthology.org/2020.repl4nlp-1.18.pdf

In [ ]:
model.bert.encoder.layer

ModuleList(
  (0): BertLayer(
    (attention): BertAttention(
      (self): BertSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (1): BertLayer

Запруним энкодер слои

In [ ]:
final_model = model

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (final_model.bert.encoder.layer[i].attention.self.key, 'weight'),
        (final_model.bert.encoder.layer[i].attention.self.query, 'weight'),
        (final_model.bert.encoder.layer[i].attention.self.value, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.2,
)

In [ ]:
for i in range(12):
    print(
        "Sparsity in Layer {}-th key weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.bert.encoder.layer[i].attention.self.key.weight == 0))
            / float(final_model.bert.encoder.layer[i].attention.self.key.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th query weightt: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.bert.encoder.layer[i].attention.self.query.weight == 0))
            / float(final_model.bert.encoder.layer[i].attention.self.query.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th value weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.bert.encoder.layer[i].attention.self.value.weight == 0))
            / float(final_model.bert.encoder.layer[i].attention.self.value.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(final_model.bert.encoder.layer[i].attention.self.key.weight == 0)
    numerator += torch.sum(final_model.bert.encoder.layer[i].attention.self.query.weight == 0)
    numerator += torch.sum(final_model.bert.encoder.layer[i].attention.self.value.weight == 0)

    denominator += final_model.bert.encoder.layer[i].attention.self.key.weight.nelement()
    denominator += final_model.bert.encoder.layer[i].attention.self.query.weight.nelement()
    denominator += final_model.bert.encoder.layer[i].attention.self.value.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th key weight: 18.59%
Sparsity in Layer 1-th query weightt: 18.67%
Sparsity in Layer 1-th value weight: 26.79%

Sparsity in Layer 2-th key weight: 18.77%
Sparsity in Layer 2-th query weightt: 18.33%
Sparsity in Layer 2-th value weight: 25.69%

Sparsity in Layer 3-th key weight: 20.08%
Sparsity in Layer 3-th query weightt: 19.58%
Sparsity in Layer 3-th value weight: 23.53%

Sparsity in Layer 4-th key weight: 18.77%
Sparsity in Layer 4-th query weightt: 18.49%
Sparsity in Layer 4-th value weight: 24.32%

Sparsity in Layer 5-th key weight: 18.40%
Sparsity in Layer 5-th query weightt: 18.36%
Sparsity in Layer 5-th value weight: 23.00%

Sparsity in Layer 6-th key weight: 18.32%
Sparsity in Layer 6-th query weightt: 17.84%
Sparsity in Layer 6-th value weight: 21.55%

Sparsity in Layer 7-th key weight: 18.54%
Sparsity in Layer 7-th query weightt: 18.06%
Sparsity in Layer 7-th value weight: 22.07%

Sparsity in Layer 8-th key weight: 18.60%
Sparsity in Layer 8-th query weigh

Предиктим на запруненной модели

In [ ]:
@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = final_model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted[0]

In [ ]:
%%time
predict('Правительство выделит 16 миллиардов рублей на поддержку клещей')
#вернулся нейтральный класс

CPU times: user 128 ms, sys: 3.36 ms, total: 131 ms
Wall time: 132 ms


0

In [ ]:
!mkdir -p output/pruning

In [ ]:
torch.save(final_model, 'output/pruning/pruned_bert.pt')

Берем и конвертируем запруненную модель

In [ ]:
onnx_inputs, onnx_outputs = export(
        tokenizer,
        final_model,
        onnx_config_for_seq_clf,
        output=Path("output/onnx_transforms/rubert_prunned.onnx"),
        opset=11)

Пробуем предиктить

In [ ]:
sess_options = nxrun.SessionOptions()
providers = [
    'CPUExecutionProvider'
]

model_ONNX = nxrun.InferenceSession("output/onnx_transforms/rubert_prunned.onnx", sess_options, providers)

In [ ]:
%%time
predict_onnx('Правительство выделит 16 миллиардов рублей на поддержку клещей')

CPU times: user 65.4 ms, sys: 448 µs, total: 65.8 ms
Wall time: 67.4 ms


0

Вес запруненной модели не поменялся, но хотя бы стало чуть быстрее на инференсе pt модели

In [ ]:
torch.save(model, 'original.pt')
!du -shc original.pt

922M	original.pt
922M	total


In [ ]:
!du -shc /content/output/pruning/pruned_bert.pt

922M	/content/output/pruning/pruned_bert.pt
922M	total


Раз пошли такие разговоры, то давайте еще torchscript

https://huggingface.co/docs/transformers/serialization#torchscript

In [49]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True, torchscript=True)

Создаем dummy input из какого-нибудь рандомного предложения

In [ ]:
text = 'Кошка села на рояль'

In [34]:
!mkdir -p output/torchscript

In [35]:
inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
traced_model = torch.jit.trace(model, [torch.tensor(x) for x in inputs.values()])
torch.jit.save(traced_model, "output/torchscript/traced_rubert.pt")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Пробуем предиктить

In [31]:
loaded_model = torch.jit.load("traced_bert.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=BertForSequenceClassification
  (bert): RecursiveScriptModule(
    original_name=BertModel
    (embeddings): RecursiveScriptModule(
      original_name=BertEmbeddings
      (word_embeddings): RecursiveScriptModule(original_name=Embedding)
      (position_embeddings): RecursiveScriptModule(original_name=Embedding)
      (token_type_embeddings): RecursiveScriptModule(original_name=Embedding)
      (LayerNorm): RecursiveScriptModule(original_name=LayerNorm)
      (dropout): RecursiveScriptModule(original_name=Dropout)
    )
    (encoder): RecursiveScriptModule(
      original_name=BertEncoder
      (layer): RecursiveScriptModule(
        original_name=ModuleList
        (0): RecursiveScriptModule(
          original_name=BertLayer
          (attention): RecursiveScriptModule(
            original_name=BertAttention
            (self): RecursiveScriptModule(
              original_name=BertSelfAttention
              (query): RecursiveScriptModule(ori

In [47]:
@torch.no_grad()
def predict_torchscript(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = traced_model(**inputs)
    predicted = outputs.index(max(outputs))
    return predicted

In [48]:
%%time
predict_torchscript('Правительство выделит 16 миллиардов рублей на поддержку клещей')

CPU times: user 90.5 ms, sys: 0 ns, total: 90.5 ms
Wall time: 90.4 ms


0

In [50]:
!du -shc output/torchscript/*

679M	output/torchscript/traced_rubert.pt
679M	total


TensorRT

https://github.com/ELS-RD/transformer-deploy